In [411]:
#########Generate customers and vechicles 
import numpy as np
import math
N=20
loc_x = np.random.uniform(0,100,N)
loc_y = np.random.uniform(0,100,N)
#print(loc_x)
#print(loc_y)
#calculate distance
distance = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        dist = math.sqrt((loc_x[i]-loc_x[j])**2+(loc_y[i]-loc_y[j])**2)
        distance[i][j] = dist
#print(distance)
dist=distance.tolist()
Q=20
M=4
demand=np.random.uniform(0,8,N)
#demand[0]=0
dem=demand.tolist()
cap=[Q]*M
d=demand

In [412]:
######Primary Route and label the customer in N ways
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] =dist
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

def get_routes(solution, routing, manager):
    
    routes = []
    for route_nbr in range(routing.vehicles()):
        
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
            routes.append(route)
    return routes


In [413]:
testdata=[]

In [414]:
 for i in range(0,N):
    
    def create_data_model():
        data = {}
        data['distance_matrix'] =dist
        data['num_vehicles'] = 1
        data['depot'] = i
        return data
    
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)
        
    routes = get_routes(solution, routing, manager)
    temp = list(set(tuple(x) for x in routes))
    testdata.append(temp)

Objective: 418 miles
Route for vehicle 0:
 0 -> 12 -> 16 -> 14 -> 9 -> 2 -> 4 -> 6 -> 15 -> 10 -> 17 -> 19 -> 13 -> 7 -> 1 -> 18 -> 3 -> 8 -> 11 -> 5 -> 0

Objective: 403 miles
Route for vehicle 0:
 1 -> 7 -> 13 -> 19 -> 17 -> 10 -> 0 -> 15 -> 6 -> 12 -> 4 -> 2 -> 9 -> 14 -> 16 -> 5 -> 11 -> 8 -> 3 -> 18 -> 1

Objective: 403 miles
Route for vehicle 0:
 2 -> 9 -> 14 -> 16 -> 5 -> 11 -> 8 -> 3 -> 18 -> 1 -> 7 -> 13 -> 19 -> 17 -> 10 -> 0 -> 15 -> 6 -> 12 -> 4 -> 2

Objective: 403 miles
Route for vehicle 0:
 3 -> 8 -> 11 -> 5 -> 16 -> 14 -> 9 -> 2 -> 4 -> 12 -> 6 -> 15 -> 0 -> 10 -> 17 -> 19 -> 13 -> 7 -> 1 -> 18 -> 3

Objective: 403 miles
Route for vehicle 0:
 4 -> 2 -> 9 -> 14 -> 16 -> 5 -> 11 -> 8 -> 3 -> 18 -> 1 -> 7 -> 13 -> 19 -> 17 -> 10 -> 0 -> 15 -> 6 -> 12 -> 4

Objective: 418 miles
Route for vehicle 0:
 5 -> 11 -> 8 -> 3 -> 18 -> 1 -> 7 -> 13 -> 19 -> 17 -> 10 -> 15 -> 6 -> 4 -> 2 -> 9 -> 14 -> 16 -> 12 -> 0 -> 5

Objective: 403 miles
Route for vehicle 0:
 6 -> 12 -> 4 -> 2 -> 

In [415]:
new=tuple()
for i in range(len(testdata)):
    temp=testdata[i]
    for j in range(len(temp)):
        new=new+tuple(j for j in temp)
        

In [416]:

testdata=new

In [431]:
#split customers to vechicles 
ans=[testdata[0][i:i + 5] for i in range(0, len(testdata[0]), 5)]
ans=ans[0:5-1]

In [432]:
newd=testdata[0][0:20]
dnew = [d[i] for i in newd]
dnew=np.array(dnew)

In [441]:

n=int(N/M)
s= [None] * M
w= [None] * M
e= [None] * M
ee= [None] * M
cn=np.arange(n)
d=dnew


In [442]:
s[0]=0
w[0]=sum(d[cn])
e[0]=math.ceil(w[0]/Q)*Q-w[0]
cn=cn+n
ee[0]=min(e[0],sum(d[cn]))
for i in range(n):
    temp=d[cn]
    if sum(temp[:i+1])>ee[0]:
        s[1]=i+n
        break

In [443]:
cn=np.arange(n)
for j in range(1,M-1):
    if max(cn) < N-1:
        cn=cn+n
    w[j]=sum(d[cn])-e[j-1]
    if w[j]<0:
        w[j]=0
    e[j]=math.ceil(w[j]/Q)*Q-w[j]
    if max(cn) < N-1:
        cn=cn+n
    ee[j]=min(e[j],sum(d[cn]))
  
    temp=d[cn]
    start=(j+1)*n
    end=start+n-1
    res=np.arange(start,end+1,1)
    for i in range(n):
        if sum(temp[:i+1])>=ee[j]:
            s[j+1]=res[i]
            break
            
    

In [444]:
print(e)
print(ee)
print(w)
print(s)

[19.72774196494349, 19.784612502125366, 16.354484030860167, None]
[19.72774196494349, 19.358342677914717, 16.354484030860167, None]
[20.27225803505651, 0.21538749787463374, 3.645515969139833, None]
[0, 9, 14, 18]


In [445]:
dnew

array([0.31811489, 3.66465645, 4.97919191, 5.85340791, 5.45688687,
       1.95101024, 6.96508934, 2.21031378, 1.5613591 , 7.255357  ,
       1.70771047, 3.05614036, 7.30664752, 4.78686424, 2.50098009,
       6.30373612, 4.14563507, 2.90761884, 3.62678577, 6.44635266])

In [446]:
###########fill the last element###########
end=N
start=N-n
#print(d[start:N])
w[-1]=sum(d[start:end])-e[-2]
if w[-1] <0:
    w[-1]=0
e[-1]=math.ceil(w[-1]/Q)*Q-w[-1]
ee[-1]=0

In [447]:
#######get dd
dd=[None]*N
for j in range(M):
    if j==0:
        dd[0]=d[s[0]]
    else:
        if e[j-1]==0:
            dd[s[j]]=d[s[j]]
        else:
            start=j*n
            end=s[j]+1
            dd[s[j]]=sum(d[start:end])-ee[j-1]

In [448]:
##########generate extended route
def extend(M,n,lst):
    res=[]
    for i in range(M):
        j=i+1
        if i==0:
            a=list(range((j-1)*n,(j+1)*n)) #####index
            res.append(a)
        elif i==M-1:
            a=list(range((j-1)*n,j*n))
            res.append(a)
        else:
            m=j
            nn=j+1
            one=list(range((m-1)*n,m*n))
            two=list(range((nn-1)*n,nn*n))
            out=one+two #######actual labeled customer 
            res.append(out)    
    return res
            
    

In [449]:
get=extend(4,5,ans) 
get

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [15, 16, 17, 18, 19]]

In [450]:
travel=[None]*M
for i in range(M):
    cn=0
    path=max(get[i])
    pos=0
    if w[i]>0:
        cn=dd[s[i]]
        then=s[i]+1      ######find the index of demand 
        if then==20:
            if round(cn,2)== round(w[i]+ee[i],2):
                travel[i]=then-1
                break
        cn=cn+d[then]
        pos=s[i]+1
        #print(pos)
        while round(cn,2)< round(w[i]+ee[i],2):
            pos=pos+1
            #print(pos)
            cn=cn+d[pos]
        travel[i]=pos

In [451]:
travel

[9, 14, 19, 19]

In [452]:
s

[0, 9, 14, 18]

In [397]:
\\

TypeError: can only concatenate tuple (not "int") to tuple

In [400]:
dist[d[0]][d[1]]

6.99053786949686

In [402]:
len(dist2[1])

21

In [463]:
########get distance and route for each veichle 
out=[None]*M
label=newd
rt=[]
for i in range(M):
    d1=0
    each=[]
    start=s[i]    
    end=travel[i]
    visit=label[start:end]
    for j in range(len(visit)):
        if j < len(visit)-1:
            d1=d1+dist[j][j+1]
    d2=dist2[0][start]+dist2[0][end]
    out[i]=d1+d2
    each=list(visit)
    each.insert(0,0)
    each.append(0)
    rt.append(each)
    
    
    
    
    
    

In [464]:
rt

[[0, 2, 9, 14, 16, 5, 11, 8, 3, 18, 0],
 [0, 1, 7, 13, 19, 17, 0],
 [0, 10, 0, 15, 6, 12, 0],
 [0, 12, 0]]

In [465]:
##########insert the depot position and recalculate the dist matrix
test=loc_x
test2=loc_y
depot=np.array(50)
new_x = np.append (depot, test)
new_y = np.append (depot, test2)
new_x=new_x[0:len(new_x)]
new_y=new_y[0:len(new_y)]

In [466]:
newdist = np.zeros((len(new_x),len(new_y)))
for i in range(len(new_x)):
    for j in range(len(new_y)):
        dist = math.sqrt((new_x[i]-new_x[j])**2+(new_y[i]-new_y[j])**2)
        newdist[i][j] = dist
dist2=newdist.tolist()            

In [471]:
dem.insert(0,0)

In [472]:
############Reoptimization 
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist2
    data['num_vehicles'] = 4
    data['depot'] = 0
    data['demands'] = dem
    data['vehicle_capacities'] = cap
    
    return data

In [473]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [474]:
data = create_data_model()

    # Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['depot'])

    # Create Routing Model.
routing = pywrapcp.RoutingModel(manager)
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0,   data['vehicle_capacities'], True,  'Capacity')

True

In [475]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = ( routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
solution = routing.SolveWithParameters(search_parameters)


    # Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)


Objective: 581
Route for vehicle 0:
 0 Load(0) ->  19 Load(1.7077104721592047) ->  2 Load(3.2690695738813744) ->  8 Load(6.895855348800705) ->  14 Load(11.04149041974316) ->  20 Load(18.34813793579153) ->  18 Load(23.327329848873422) ->  0 Load(23.327329848873422)
Distance of the route: 149m
Load of the route: 23.327329848873422

Route for vehicle 1:
 0 Load(0) ->  7 Load(5.853407909018795) ->  16 Load(11.310294776186655) ->  11 Load(17.756647440216632) ->  1 Load(19.70765767708785) ->  0 Load(19.70765767708785)
Distance of the route: 120m
Load of the route: 19.70765767708785

Route for vehicle 2:
 0 Load(0) ->  6 Load(2.5009800921129575) ->  12 Load(4.711293869052402) ->  9 Load(11.676383212437537) ->  4 Load(17.980119332877724) ->  0 Load(17.980119332877724)
Distance of the route: 151m
Load of the route: 17.980119332877724

Route for vehicle 3:
 0 Load(0) ->  13 Load(3.6646564517388134) ->  5 Load(3.9827713457879623) ->  3 Load(7.038911705800923) ->  10 Load(9.946530546734174) ->  15

In [476]:
out

[633.8986445396627, 393.07863774795146, 373.5898651428695, 65.14484966424256]